# Pump systems

A pump is used to move fluid through a pipeline.




<img src="data:img/png;base64,iVBORw0KGgoAAAANSUhEUgAAASwAAAB6CAYAAAAF3W4rAAAAAXNSR0IArs4c6QAAAAlwSFlzAAAXEgAAFxIBZ5/SUgAAEx1JREFUeAHtnQuUVMWZx6tvv+YBaBAJCDi8dGQHXTCDPKJEUUSTKGQJOagkMREX3D3J8cSNG41y9mGO0bzY7LIxOZqQXY4YHxtwg0qIawQEBHkpAQYwCIiMQgaEYfp17+39/re7ZpqhhxmgZ+hb/S/9qNt161bV9yv6z1e37yOgWlKlbAZbPvpq66ivRsvBkgAJnBGBQPaoEZJvPKMWiuOgWTKMXxTHUDgKEihaAkMikchDlmX5LjBxXddJJpOPhrJoK6uqqhrr6uq6FS3qNgY2ffr02KJFi3q0sZvFJEACLQTG9OrV67ZZs2ZFW4r8sfXkk08m9u3bt1wLlhLVTUejvvNDBYO++8fCH39DOEoTCaT79euXnDNnju++6MuWLUuJYCnLxFmhTyRAAmYSoGCZOa/0igSMJEDBMnJa6RQJmEmAgmXmvNIrEjCSAAXLyGmlUyRgJgEKlpnzSq9IwEgCFCwjp5VOkYCZBChYZs4rvSIBIwlQsIycVjpFAmYSoGCZOa/0igSMJEDBMnJa6RQJmEmAgmXmvNIrEjCSAAXLyGmlUyRgJgEKlpnzSq9IwEgCFCwjp5VOkYCZBChYZs4rvSIBIwlQsIycVjpFAmYSoGCZOa/0igSMJEDBMnJa6RQJmEmAgmXmvNIrEjCSAAXLyGmlUyRgJgEKlpnzSq9IwEgCFCwjp5VOkYCZBChYZs4rvSIBIwlQsIycVjpFAmYSoGCZOa/0igSMJEDBMnJa6RQJmEkgZKZb9IoEipJAmYyqSqyXWEV2hHHJD4ntFTueLWPWBgEKVhtgWEwCBSIw1AqFvhEJRyfHY8cHSJttrWrS0bKKettxljipxH9Ivc0F6t+oZihYRk0nnSkyAuMClvVH17bDcdt2ZWz5xCpmhcLbIpHytG0nhohYzZR6XxebKrZIjCmHQD6AObu5SQIkcBYEatKuGw5YweOWFXxK2lkgtkoFArlLv3LXTl2ZSiUHqYC1wgqGG6SOpSxrxFn0a+yhjLCMnVo6VgQEnhLh6SUi9G0nFb+7eTzpdPOm3pD9PWX7Fom2Gi0r9Ljr2o/qfcxbCFCwWlhwiwQKTcBVrvuo48Yfk4avEEPUNEhZwX4SdX0anaXTznrlOO/J5h6xzRJtbZDcFmPKQ4CClQcKi0igwARw/mpT1pRynfPSrnMk28cUyeuy28zaIcBzWO0A4m4S6AQCgZw2T14f5uzk5okEKFgn8uAnEugKAhSsM6RMwTpDcDyMBM6CQO73DstFpg4SyAXXwUNYjQRKngAipEliXxQrF8NnbbLZbjrTCAvH4TuLq+SniU0Uy21LPpqdKFhmzy+9KzwBCEQoGK24zwpFngkEQ3+xwpGnpexmlIsFxVDnVEKSu6+9c1ioi+9pROyzwXDZQunzkPS9UMbwLSlDn7ntyUdzE38lNHdu6VnnEIB4RAOBQGj41PuDfS+pLd+zdvG0P69aNDkVP56S6xSedZ3UAqnzhhiWexCk1qKUGyi0XhJq8UEdbF8t13HNkItNp4XLuoUHj5sSqRp9a/DA9nVqy4tz8f0NZ9t3srlk5iYKlrlzS886hwCEBCIRDIie9BxYoy4cdHlw3O0Pl/9l1/ryd1ct+mrdqkVfdtPuUTuVWCDXYSH6wrVVucKlRUmKTxAzlKP9kXLx6B1WODRD8m7V46YEh4ybHO41tFYlHVfZYge2r8Wxeiy4bkuLI8qNTRQsY6eWjnUiAVn2pSEWIlnyn8hMIGCpi4aNVgMvHxe+afZj6v13Vly4Zflzf7dt1ZK/t4LB+mS8ab5UXyi2wztM/sgmCBnaqha7LVJWcafrOp8cNvbz6eHXTov2r7lapdIBlbIdlRBDkujOy+UPHAdrLtA7TM0pWKbOLP3KJYAvNM4tFSLhO4PzSVZagiP8l9GLtKca0BIrGFJDP3W9qhlzY9S1k2rXut9fvOHVZ769de2yB8ORsncTscbncwYyO1re7YupZHzwsKsm2rU33F42RI61wlGVSDleRBXwhAouBGTFqfv0WoBYYSwpMQyk9fJSik5IFo73c6Jg+Xn2OPZ8BD4lhZPLK3pc76adwa5jV7iuazl2qlu+ymdalpZlGaIrTyakEUQ9WgsgCjoKConwXDF+iqqdMLXciTeq1UvmVz//xEPflXF5XQfDkQen3PWwGvv5rykrUhFKyUMdErat3GxjGYFCVQhNpt1AVnPkavnxUliPvR1Nmzdvbk/UOtrUOalHwTon2NlpJxC4KVpW+UNZTl1Se+3kdM2oCdE+Ay5VFd3PU03HPlb1e+rU7u3rnfUr/jdx7EiD49jJn8oYfi6WPM2x4PzVebIE/JXko7RQQV+8lZqXZyIhtIv98aZjatOal9SbSxfEdm5aGYlEy3ckHXspdodCkUnP/ueDl2xc9VLyqhtnlF825mYVjFaqtJtRJRyP2EqqZk1kyxMzLEWt1VJ4l1iTGPxoT4y+UFNT8wOpV1Dxlva6LFGwugw1O+okAlFZZv1KvthTb/3aA5FJ07+p5EF4J3VVPfIa9ZnAzOCd3/lZxcaVv1NPz/2H+xo+2n97MhHD9VQ7Tzqg7QJvCSZnsFIti7OMuEBU9JadjKtda19VG1/9TWzHW3+IhMPR/Yn48d9Ls39Ixo/vldz7VU+Wh8/K9oAdG16/cc/WtyamUvf0rR51gz1iwvSygSOvV+kA9FGLlLcJFUQJcojUQTE8rgbbmZNcstFGOhoOh/UQ26hS3MUUrOKeH47u1AR6RaIVyy4aNKz6Wz9aHOnRs7dIRkA5rg40sl9sLzpBQ/iuBtQV4z6rho+5qeyFJ+YMeGXh3I0Sbd0iO15DjY6mTLyTiX4QBSHJslN9sGWV2r7yhcTONS9ZgWDwcCrehHZfTzj2nyVPiOGRyDExrAkxICgSBGyBCNoLkldtW/PKtbs2Lb827TifuHTM59LVV0+N9Bk2VnZZ3rIz05t8LMFEwSrBSTfE5Yj8ovZSzVUT/2r2vyyIhMIRuYIgE/PoLzQ+YRs5kt7O7A+oqfc8EupTVR2a//3ZvxOxGSVVtnoVT+MPOT+mDu7coPa++WLq3TUvyoo0FXNse7lKO6/JqfA6aSpXpHQkhDJEQ1BWnDjH1fIIC5F/LLZDhO7XkldvXbn4uro1S66Ra7HKh4y91aoaPTl8wZArZVdpJgpWac67770OR8vmXdhvyPCvPzw/kg4EvZ/9vaVS88mkPC7qfd45INkvkdGoibepD/fvjr7y9I+W2YnYZVJ6LM+RJxeJ1Ox9c7G7bcm8RCreKOfI3ZVpx0Y0hWexY3mmIymIFCIqfEY5ftFDdKWXb/j1slEMkRZeUgGDeCE/LMK3yUk58+TRySPqlj87YceK5z8tF5BaFRf0i8r+kksUrJKbciMcHuvazp2zvvdcCD//O5kASkIpBCwSP3l5Pj/1vkyMpX/Wu/mrDwa3rF7a8/1dm/7VtlP35juyVZlEUonNh9+vi6ftxB9l31tiiJogSBAnbRApmBYpvQzEQPWosY1y1MHxOIEO0YRg4XwZhAkCtlwEca0cVJZobKiV82DjZfG7RcrRjm5LNs1OFCyz59dI78Jlld+75ta7rPN79885X6Vdbfu7G5DrATJBVqZObsD1N994vGzuNyfeI608InZIt5Ynx8EiWI682cY5X7ahfhCaXMHSQgUh0obj8g1Ol6EexEsLF9rE9xORF6I0LBdhELLXRChflgPx/Hfv5L3kJZEoWCUxzUY52d9OxsZfN+1eC+esTpUy57Dwy122HjLIS/YjTpbra6Yurq5VA6qvtPdsXXuH1Pi3U7SrRQXLOCR8hrjoSEoLF8q1mKDHbK+ydXLS+3AMEj5DwHB+CwbRQtSGaAsG0ULbKIPA4TjdhmyamyhY5s6tqZ7dPODSK2PR7j274VaVHP05yV/8eKejqMxOXRs59snFn55oZXRi5HVfqqjfvXWaXGpwKsGCMECUcHIckQ8O1lGU3kYdbbLZ4aRFB2KEQWYGlmkfooj+sEzEeS/sRxkM9fSxsmluomCZO7dGeia/lo0d/NfjKx0okVYj5EitFQoRGMraisS8ffJdRy5pwGWjlNywPML70PYfEAdENVpU0Hlra/voju/JOuUdgL60eKFvRF1IKIPl1kW5sSkzU0rhdoZ1wWBQQyl6h+Xn5ICE83r89TLgwzmD1uW6iJ81iUzuZx7lwVAkiPv1WtKJ31cs88LRcvXdZ3Y1L/lQ9yR9Q5mYhhGPHVWPTBuEqueJHcXGKZI+7MTOT3FAgXeh/9Pt+46+ffs+NXPmTN/9wjh//vzkvn37ZmvoYDlALPdvAcqKOQ2WwV2UHeBHkmPyWk8gP2cBZTPf84hEKxddN+M7l11Se0NzZKRd1Bdw4rO8Rktd0GfgCYKk67WV45qqf57cG7v7iuEfQdPSILnS/QHhVKgbwbuMj/wj5KRSqcdzBWpfl/VemI52F6YZtuIrAkGrIVTWXfXoM0SGDf3NDTT0dubfYTztoHmZ2G5tOWse0+fRvUsLfIWlg4PdLV/6v+1g3aKslitYRTlADooEcgnYqfiOhg/fG4fbbzJLvMylCqjT+rIF7zh9fiu3kTa2D9e/p+TNy03yMtP2loNttMDiziagT951dj9snwQKQsBNpda89/aKRgiWPNXTuw4LOczOliGHOXKyXee52y37db1MO3u3rVXBcHRTQQbKRjqFACOsTsHKRjuRwMv1uzbOO3L4oCrv3lOiqhMf5aKvq0L/J51kl5WiLss3vq0rXmhKxRqfy7ePZcVBgBFWccwDR9FxAnslClr11os/c72oSR6kh9wzuUenOZdtXFjqZHNs41noOse2LQ/L0/n+uo2qfudGXKD5dMeHwppdTcAEwZok0Er39vWu/htz7vrrL11/Gd3LTcoPvP3yk+7HB/eLAMkFUVnB0ss/veRrnaMuypB7x0m4pfOV//1PcXkg3s+lefzizFSkBEwQrBnC9sYi5cthFY7AMGnqgWxzb8jZ8QVLf3J3LJGIyc3PEknBIFw6z4qYFjPkrQUN58FQtn7xvzuH9v7piDwa5qHCDZctdQYBEwQLt0UglGcymwDmGHPtJTcVn91w4N3t//fEvclYLJ59YYO8WcZ7cQNe3pB5y4zOcRuP3kaeMVfVvfFbteG3c5NOMo5/9HC7DVMREzBBsHADqG+fUV3EfzeKbWiVMiDMtU4JNxm7af/br+1Y+tj02PGPD0m0hCgLUROiKVnuSbSlc28b57BQJjlem7Xhf37svPHU/U3yQojJ0ug7umHmxUvAd1e85kF5jZThVoolefaxyBwCY8WVS8X+K8el4yI28xPHDg+ve/XXQwNWKHj+gBq5ICsovwZmBEvnECr53xOsA39aqV6fNyv+wZbX6+U1XOOlvTdz2uRmERPIXBJcxAPswND+UerUik3rQF1W8S8BzDPuef1SGy7cEoqU/1A0qarfyEmB3tWjI916V6lweXeFdwM2NXygGva84+5btyQWP/KRvJTG+YGccX9c2sJjYZhIoMsIfEV6Wt5lvbGjc0Xgx9LxTzvQOSKx74fKKtdZ4bKDgWDouBWKHAlFu9VZ4QiiMwgeHoTH5EMCJlw4ule448ZtJrMJXCzuremAi6ulzmo7jkdHZRLuOESUxeR/Apb/XfDeTALB8t0jMwxg35UuDJXOtndlh+yLBDqLAJ400d6D1zqrb7bb+QT0I4I/2fldsYdiJmBChAW+L4t9pphBc2xnRWC0HL1N7MOzaoUH+56ACb8SYhIuF1spVgq/+CCaxO1Ih8XwpuBxYqYnXIN1t9hC0x2lf6cmYIpgwcv+Yt7jIk/tsu/3zhIPPhBbL3a/2L1ipic8WW+H6U7SPxIwkQCWvpvFfil2n4kO0icSIAFzCOCqfkdsv9gEc9yiJyRAAqYSwAsScHlRP1MdpF8kkI+AKb8S5vPN5LI94hzeT3fAZCfpGwm0JkDBak3EH58hVHjQnOuP4XKUJFAYAhSswnDs6lYOSocwJhIoKQIULH9ONx40d8SfQ+eoSeDMCVCwzpzduTzymHTe/NbPczkQ9k0CXUmAgtWVtAvXFx49kChcc2yJBPxBgILlj3lqPUoIFoyJBEiABIqeQA8ZIa/BKvpp4gBJgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARIgARI4HQI3CaVd4t95XQOYl0SMIEAH+Dnv1nsI0MeKIaciQRKigAFq6Smm86SgL8JULD8PX8cPQmUFAEKVklNN50lAX8ToGD5e/44ehIoKQIUrJKabjpLAv4mQMHy9/xx9CRQUgQoWCU13XSWBPxNgILl7/nj6EmgpAhQsEpquuksCfibQMjfwy/p0ePWnDGtCMyRz1talfEjCZAACZwzAtOkZ1csnceuP2ejYsckQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkQAIkUHAC/w8hEHXusxZJAAAAAABJRU5ErkJggg==" width=300> 


* The flow rate is controlled by a valve.
* The pump supplies pressure at various flow rates. 
* If we divide the pressure by the fluid density and gravity, which are both constant, we get units of length, which we call a **head**.
* So, we can plot **head** versus flow rate. Here's an example of a ***pump curve***
    * We'll consider the 12.75 inch pump size (top curve).

<img src=pump.png width=600 align=center>
* The flow through the system is related by a mechanical energy balance:

$$
\frac{\Delta P}{\rho g} + \frac{\Delta v^2}{2g} + \Delta z = h_p -\frac{fLv^2}{2Dg} - \frac{Kv^2}{2g}
$$
* Here, $z$ is height, $f$ is the pipe friction factor, $L$ is the pipe length, $D$ is the pipe diameter, and $K$ is a valve coeficient.
* We'll assume the tanks are open to the atmosphere, have no vecocity at the top surface, and are at the same height, so that the left hand side is zero. We're assuming no other friction losses besides the pipe and the valve. Rearrange this to
<font color=blue>
    
$$
h_p = \frac{fLv^2}{2Dg} + \frac{Kv^2}{2g}.
$$
</font>

* The system operates where the **operating curve (equation above)** and the **pump curve** in the figure intersect.
    * That is, the system can operate anywhere on the operating curve defined by the equation above, (that is, any head/flow combination given by that equation), **but because the flow is driven by a pump, we are restricted to the points that also lie on the pump curve.**
    * We adjust the intersection point, that is, the operating point of the system, by opening or closing the valve (changing $K$). This will effectively pivot the operating curve left or right about the origin.

Consider a pump system with a pipe with the following parameters:
* L = 35 m
* f = 0.015
* D = 0.1

The operating point can be visualizaed by opening/closing a valve (changing $K$).

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import pint; u = pint.UnitRegistry()
from io import BytesIO
import ipywidgets as wg
from IPython.display import display

#------------ 

Qp = np.array([1.16, 40.8, 80.5, 121, 161, 201, 240, 281, 321, 359, 401, 440, 480, 519, 560, 600, 640, 680, 720, 760, 800, 827])*u.gal/u.min
hp = np.array([76.2, 76.2, 76.3, 76.0, 76.1, 75.6, 75.2, 74.7, 74.1, 73.0, 71.3, 69.7, 68.0, 65.7, 63.3, 60.5, 57.3, 53.7, 49.1, 43.7, 36.7, 30.9])*u.ft
Qp.ito(u.m**3/u.s)
hp.ito(u.m)
Qp = Qp.magnitude
hp = hp.magnitude

#------------ 

def operatingcurve(Q, K):
    L = 35
    D = 0.1
    f = 0.015
    g = 9.81
    
    v = 4*Q/np.pi/D**2
    
    return f*L*v**2/2/D/g + K*v**2/2/g

#------------ 

pumpcurve = interp1d(Qp, hp, fill_value='extrapolate')

#------------ 

plotpic = wg.Image(layout=wg.Layout(width='60%'))

def get_operating_point(K):
    
    def F(Q,K):
        return operatingcurve(Q,K)-pumpcurve(Q)
    
    Q = fsolve(F, 0.02, args=(K,))[0]
    h = operatingcurve(Q,K)
    print(f"Q={Q:.3f} m^3/s,\nh={h:.1f} m")
    
    Qpts = np.linspace(np.min(Qp), np.max(Qp), 100)
    plt.plot(Qpts, operatingcurve(Qpts,K))
    plt.plot(Qpts, pumpcurve(Qpts))
    plt.plot([Q], [h], 'o', markersize=10)
    plt.ylim([0,30])
    plt.xlabel(r'Q (m$^3$/s)')
    plt.ylabel('h (m)')
    
    plotpicdata = BytesIO()
    plt.savefig(plotpicdata, format='png')
    plt.close()
    plotpicdata.seek(0)
    plotpic.value = plotpicdata.read()
    
#------------ 

userinput = wg.interactive(get_operating_point, K=(0,200,5));
box = wg.VBox([userinput, plotpic])
display(box)